<a href="https://colab.research.google.com/github/samyff12/LATAM/blob/main/IMAE_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Actualizacion IMAE LATAM**

### Paquetes y meses

In [1]:
from bs4 import BeautifulSoup
import pandas as pd    # ! pip install --upgrade xlrd  por si no se actualiza pandas
import numpy as np
import requests
import urllib.request as urllib2
import re
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
!pip install XlsxWriter
from google.colab import files


#!pip install kora -q
#from kora.selenium import wd

# Selenium
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
# Argumentos de Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)
# En caso de contar con archivos en formato pdf: 
# !pip install pymupdf            
# !pip install tabula-py
# import tabula as tb
# import fitz  # this is pymupdf

In [ ]:
! pip install --upgrade xlrd
! pip install --upgrade openpyxl
##html = requests.get('http://www.secmca.org/chart/?parent=Producci%C3%B3n&scid=2&cid=1&scsid=0&son=%C3%8Dndice%20Mensual%20de%20la%20Actividad%20Econ%C3%B3mica&url=11/14/3-13-50-17-4-52-161/39/265/19-258/444/' +
 #                   '20211-20212-20213-20214-20215-20216-20217-20218-20219-202110-202111-202112-20221-20222-20223-20224-20225-20226-20227-20228-20229-202210-202211-202212' +
 #                   '-&all_vars=2|IMAE,%20variaci%C3%B3n%20interanual')
#soup = BeautifulSoup(html.text,'lxml')
#table = soup.find_all('p', class_='text-right')
#tablec = soup.find_all('th', colspan='1')

#indicadores=[]
#paises =[]
#for i in range(len(table)):
 # ind = float(table[i].text)
#  pais = tablec[i].text
#  indicadores.append(ind)
 # paises.append(pais)

#imae_latam =  pd.DataFrame(indicadores,index = paises,columns = ['IMAE'])
#print(f'IMAE a {fecha}')
#imae_latam

In [3]:
meses = [0,[1,'Enero'],[2,'Febrero'],[3,'Marzo'],[4,'Abril'],[5,'Mayo'],[6,'Junio'],[7,'Julio'],[8,'Agosto'],
         [9,'Septiembre'],[10,'Octubre'],[11,'Noviembre'],[12,'Diciembre']]
mes3 = {'ene':'Enero','feb':'Febrero','mar':'Marzo','abr':'Abril','may':'Mayo','jun':'Junio','jul':'Julio',
        'ago':'Agosto','sep':'Septiembre','oct':'Octubre','nov':'Noviembre','dic':'Diciembre'}
meses_dict = {0:'none',1:'Enero',2:'Febrero',3:'Marzo',4:'Abril',5:'Mayo',6:'Junio',7:'Julio',8:'Agosto',
         9:'Septiembre',10:'Octubre',11:'Noviembre',12:'Diciembre'}

header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}

### import calendar

# d = dict((v,k) for k,v in enumerate(calendar.month_abbr))
# d

## Para obtener las series de todo el año 2021-2022


### **SECMCA**

#### Publicaciones disponibles las fechas: 

#####    **El Salvador**
2. 28 Feb - Lunes
3. 31 Mar
4. 29 Abr
5. 31 May

#####    **Panama**
- 21 Mar (se publica Dic y enero )
- 19 Abr
- 19 May
- 20 Jun
- 19 Jul
- 18 ago


#####    **Honduras**
-. Preliminar 20 de cada mes, con 50 días de rezago y
revisado 30 días después

In [4]:
html2 = requests.get('https://www.secmca.org/chart/?parent=Producci%C3%B3n&scid=2&cid=1&scsid=0&son=%C3%8Dndice%20Mensual%20de%20la%20Actividad%20Econ%C3%B3mica&url=11/14/3-13-50-17-4-52-161/39/265/19-258/81/20171-20172-20173-20174-20175-20176-20177-20178-20179-201710-201711-201712-20181-20182-20183-20184-20185-20186-20187-20188-20189-201810-201811-201812-20191-20192-20193-20194-20195-20196-20197-20198-20199-201910-201911-201912-' +
'20201-20202-20203-20204-20205-20206-20207-20208-20209-202010-202011-202012-20211-20212-20213-20214-20215-20216-20217-20218-20219-202110-202111-202112-&all_vars=1|IMAE').text

In [5]:
# INDICE VERSION
html1 = requests.get('http://www.secmca.org/chart/?parent=Producci%C3%B3n&scid=2&cid=1&scsid=0&son=%C3%8Dndice%20Mensual%20de%20la%20Actividad%20Econ%C3%B3mica&url=11/14/3-13-50-17-4-52-161/39/265/19-258/81/' +
  '20171-20172-20173-20174-20175-20176-20177-20178-20179-201710-201711-201712-20181-20182-20183-20184-20185-20186-20187-20188-20189-201810-201811-201812-20191-20192-20193-20194-20195-20196-20197-20198-20199-201910-201911-201912-' +
  '20201-20202-20203-20204-20205-20206-20207-20208-20209-202010-202011-202012-20211-20212-20213-20214-20215-20216-20217-20218-20219-202110-202111-202112-20221-20222-20223-20224-20225-20226-20227-20228-20229-202210-202211-202212' +
  '-&all_vars=1|IMAE').text
soup1 = BeautifulSoup(html1,'lxml')

tabpais = soup1.find_all('th', colspan='1')
table = soup1.find_all('p', class_='text-left')


In [6]:
fechas = []
for i in range(len(table)):
  x = re.sub(" ","",table[i].text)
  fechas.append(x.replace("\n","").replace('Setiembre','Septiembre'))

paises = [tabpais[i].text for i in range(len(tabpais))]
fechas[-1]

'2021-Diciembre'

In [7]:
#INDICE
table1 = soup1.find_all('p', class_='text-right')
matriz = np.array([])
for i in table1:
  matriz = np.array(np.append(matriz,i))
matriz = np.where(matriz=="--", np.nan, matriz).astype(np.float)
matriz = matriz.reshape(-1,7) # 7 paises
imaeLA = pd.DataFrame(matriz,index=fechas,columns=paises)
imaeLA.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá
2021-Agosto,106.40,113.32,130.02,257.01,154.33,188.80,322.86
2021-Septiembre,109.68,114.29,129.23,251.33,150.08,181.01,317.43
2021-Octubre,112.01,110.98,131.39,267.44,161.08,192.05,366.56
2021-Noviembre,117.14,117.78,135.38,265.80,168.49,204.87,370.39
2021-Diciembre,120.31,NaN,140.26,NaN,NaN,212.34,NaN


import re
tabpais = soup.find_all('th', colspan='1')
table = soup.find_all('p', class_='text-left')
fechas = []
paises =[]
for i in range(len(table)):
  x = re.sub(" ","",table[i].text)
  fechas.append(x.replace("\n",""))
  pais = tabpais[i].text
  #indicadores.append(ind)
  paises.append(pais)
fechas[-1]

table1 = soup.find_all('p', class_='text-right')
matriz = np.array([])
for i in table1:
  matriz = np.array(np.append(matriz,i))
matriz = np.where(matriz=="--", np.nan, matriz).astype(np.float)
matriz = matriz.reshape(-1,7) # 7 paises
imaeLA = pd.DataFrame(matriz,index=fechas,columns=paises)
imaeLA

### **Argentina**


#### Publicaciones disponibles las fechas: 
1. 28 sept 
2. 21 Oct
3. 23 Nov
4. 23 Dic

In [8]:
url = "https://www.indec.gob.ar/ftp/cuadros/economia/sh_emae_mensual_base2004.xls"
urllib2.urlretrieve(url, "test.xls")
data = pd.read_excel('test.xls',header=2)[2:]
data1 = data.iloc[:,:4].dropna(subset = [data.columns[1]]) # data.columns[1] es la columna de los meses
data1.reset_index(drop=True,inplace=True)
data1[data1.columns[0]] = data1[data1.columns[0]].ffill() # forward fill # para los años   data1.columns[0] es la columna de los años
data1 = data1.iloc[11:,:3]

In [9]:
data1['fecha'] = data1[data1.columns[0]].astype(int).astype(str) +"-"+ data1[data1.columns[1]]
data1.set_index(data1.fecha,inplace=True)
data1.iloc[:,2].tail(1)

fecha
2021-Noviembre    145.394221
Name: Índice Serie Original\n2004=100, dtype: float64

In [10]:
imaela = pd.merge(imaeLA,data1.iloc[:,2],how='left',left_index=True,right_index=True)
imaela.columns = [*imaela.columns[:-1], 'Argentina']
imaela.tail(1)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina
2021-Diciembre,120.31,NaN,140.26,NaN,NaN,212.34,NaN,NaN


### **Brasil**

#### Publicaciones :
1. 15 sept
2. 15 Oct
3. 16 Nov
4. 15 Dic

In [11]:
urljson = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.24363/dados?formato=json"
urllib2.urlretrieve(urljson, "brasil.json")
bra = pd.read_json('brasil.json')
bra.rename(columns={'valor':'Brasil'},inplace=True)

bra['Brasil1'] = bra.Brasil.pct_change(12)*100 # calculo tasa de crecimiento
bra['date'] = pd.to_datetime(bra['data'],format='%d/%m/%Y') # transformamos a formato fecha
# Creamos indice
bra['fecha'] = pd.DatetimeIndex(bra['date']).year.astype(str) +"-"+pd.DatetimeIndex(bra['date']).month.astype(str)

nombre_mes = [meses[i][1] for i in pd.DatetimeIndex(bra['date']).month.tolist()] # obtenemos nombre del mes
bra['fecha'] = pd.DatetimeIndex(bra['date']).year.astype(str) +"-"+ nombre_mes # creamos indice
bra.set_index(bra.fecha,inplace=True) #reemplazamos indice


bra.tail(1)

,data,Brasil,Brasil1,date,fecha
fecha,,,,,
2021-Diciembre,01/12/2021,141.46,1.303351,2021-12-01,2021-Diciembre


In [12]:
imaela = pd.merge(imaela,bra['Brasil'],how='left',left_index=True,right_index=True) # combinamos la data

In [13]:
#    imaela

### **Mexico**

#### Publicaciones disponibles las fechas: 
1. 27 sept 
2. 25 Oct
3. 25 Nov
4. 24 Dic

In [14]:
url = "https://www.inegi.org.mx/contenidos/temas/economia/cn/igae/tabulados/ori/IGAE_1.xlsx"
urllib2.urlretrieve(url, "test1.xlsx")
mex = pd.read_excel('test1.xlsx',header=3,nrows=3)
mex = mex.T.reset_index(drop=True) # transponemos y estandarizamos el indice
mex.drop(0,inplace=True) # borramos primera fila
mex[2] = round(mex[2].astype(float),2) # leyendo tasas de crecimiento en el formato numerico y redondeando
mex[0] = mex[0].ffill()

In [15]:
mex[1] = mex[1].replace(['oR','oP','lR','lP','eR','eP',' '],['o','o','l','l','e','e',''], regex=True) #limpiar superindices "Revisado" "Preliminar"
mex[0] = mex[0].replace(['P','R',' '],['','',''],regex=True) # limpiar preliminar de fecha
mex.index = mex[0].astype('str') +"-"+ mex[1] #creamos indice para base de datos
mex.columns = [*mex.columns[:-1], 'México'] # nombramos columna
mex.dropna().tail(1) # mostrar ultimo valor no nulo

,0,1,México
2021-Noviembre,2021,Noviembre,112.75


In [16]:
imaela = round(pd.merge(imaela,mex['México'],how='left',left_index=True,right_index=True),2) # combinamos las series

In [17]:
imaela.tail()

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Brasil,México
2021-Agosto,106.40,113.32,130.02,257.01,154.33,188.80,322.86,141.08,142.17,107.14
2021-Septiembre,109.68,114.29,129.23,251.33,150.08,181.01,317.43,140.91,138.86,104.18
2021-Octubre,112.01,110.98,131.39,267.44,161.08,192.05,366.56,140.53,138.07,108.41
2021-Noviembre,117.14,117.78,135.38,265.80,168.49,204.87,370.39,145.39,139.76,112.75
2021-Diciembre,120.31,NaN,140.26,NaN,NaN,212.34,NaN,NaN,141.46,NaN


In [18]:
## link excel
#https://www.inegi.org.mx/contenidos/temas/economia/cn/igae/tabulados/ori/IGAE_1.xlsx       indice

#https://www.inegi.org.mx/contenidos/temas/economia/cn/igae/tabulados/ori/IGAE_2.xlsx       crecimiento

### **Perú**

#### Publicaciones disponibles a fecha:
 
2. 21 Feb

In [19]:
url = 'https://www.inei.gob.pe/media/principales_indicadores/CalculoPBI_101.zip' #cambio de 99 a 100 (sep oct)

In [20]:
urlpe= 'https://www.inei.gob.pe'
wd.get(urlpe)
souppe = BeautifulSoup(wd.page_source,'lxml')
link_html = souppe.find_all('a',class_='btninei',title="documentos adicionales") # para encontrar el nombre del link de indice peru
for a in link_html:
  link = a['href']
  print(link)

urlpe = 'https://www.inei.gob.pe/' + link

/media/principales_indicadores/CalculoPBI_102.zip


In [21]:
zip, _ = urllib2.urlretrieve(urlpe)
with zipfile.ZipFile(zip, "r") as f:
    f.extractall('/content')

In [22]:
indicador = [f.namelist()[i] for i in range(len(f.namelist())) if 'Indicador Mensual' in f.namelist()[i]][0] # el 0 es para que elija el string de la lista
df = pd.read_excel(indicador,sheet_name=1,header=3)[1:] # el indice esta en la segunda hoja del excel

In [23]:
indicador

'Indicador Mensual de la Producción Nacional 12 2021 r.xlsx'

In [24]:
pe = round(df.filter(like='Índice Global'),2)   # regex='spike|spke' se puede usar regex para en caso de que escriban las columnas mal
pe['YM'] = df['Año y Mes'].astype(int).astype(str)

#  IG.1 Mensual   IG.2 Acumulado  IG.3 Anual
pe['fecha'] =  pe['YM'].str[0:4] +'-'+ pe['YM'].str[4:7].astype(int).map(meses_dict)  # fecha
pe.set_index('fecha',inplace=True)

pe.rename(columns={pe.columns[0]:'Perú',pe.columns[1]:'Perú_i',pe.columns[2]:'Perú_a'},inplace=True)

In [25]:
imaela = imaela.merge(pe['Perú'],how='left',left_index=True,right_index=True)

### **Chile**

#### Publicaciones disponibles las fechas:
 
-   1ero o 2do día de cada mes

In [26]:
url = "https://si3.bcentral.cl/estadisticas/Principal1/excel/CCNN/IMACEC/Excel/IMACEC.xlsx"
urllib2.urlretrieve(url, "chile.xlsx")
df = pd.read_excel('chile.xlsx',header=5,keep_default_na=False)

In [27]:
df.columns = df.columns.str.strip() # eliminar espacios innecesarios
chile = df.loc[1:,[df.columns[0],df.columns[1],'Imacec']][:-10] # excluimos las ultimas 10 filas (pie de pagina)
chile.rename(columns = {'Imacec':'Chile'}, inplace = True) # renombramos
chile['Chile1'] = pd.to_numeric(chile.Chile).pct_change(12)*100
###
chile[chile.columns[0]] = pd.to_numeric(chile[chile.columns[0]], errors='coerce').ffill().bfill().astype(int)
 # las columnas del excel contienen string en blanco por lo que se transformaron a numero antes de hacer el forward fill
 # el backward fill es para asegurar que no quede NaN por detras y dificulte la creacion del indice
###

chile['mes'] = chile[chile.columns[1]].map(mes3).bfill() # creando mes y usando el bfill por el mismo motivo explicado arriba

#Creamos el indice
chile.index = chile[chile.columns[0]].astype('str') +"-"+ chile.mes

chile.tail(1)

,,Unnamed: 1,Chile,Chile1,mes
2021-Diciembre,2021,dic,135.293629,10.118327,Diciembre


In [28]:
df.head()
df.columns = df.columns.str.strip()

In [29]:
 pd.to_numeric(chile[chile.columns[0]], errors='coerce')

2013-Enero         2013
2014-Enero         2014
2015-Enero         2015
2016-Enero         2016
2017-Enero         2017
                   ... 
2021-Agosto        2021
2021-Septiembre    2021
2021-Octubre       2021
2021-Noviembre     2021
2021-Diciembre     2021
Name: , Length: 118, dtype: int64

In [30]:
imaelatam = round(imaela.merge(chile['Chile'],how='left',left_index=True,right_index=True),2)
imaelatam = imaelatam[~imaelatam.index.duplicated(keep='last')].reindex(imaela.index) # hay duplicados, los eliminamos y reindexamos

In [31]:
#  imaelatam

### **Ecuador**

#### Publicaciones disponibles las fechas: 
1. 14 sept    -    Julio
2. 13 Oct    -     Agosto
3. 15 Nov    -     Sep
4. 24 Dic    -     Oct

In [32]:
url = "https://contenido.bce.fin.ec/documentos/PublicacionesNotas/Catalogo/IEMensual/Adelantos/IEM-452-e.xlsx"
urllib2.urlretrieve(url, "Ecuador.xlsx")
df = pd.read_excel('Ecuador.xlsx',header=7,keep_default_na=False)[1:13].reset_index()

ecu = (df.set_index(['Unnamed: 0','index'])
.stack()
.reset_index()
.rename(columns={'Unnamed: 0':'mes','level_2':'Year'})).sort_values(['Year','index']) # ingenieria de base de datos

# Elaboracion de indice
ecu['Mes'] = ecu.mes.str.lower().map(mes3)
ecu.insert(3, 'fecha', ecu.Year.astype(str) + '-' + ecu.Mes)
ecu.index = ecu.fecha

# Variacion interanual
ecu.rename(columns={0:'Ecuador'},inplace=True)
ecu['Ecuador1'] = pd.to_numeric(ecu['Ecuador'],errors='coerce').pct_change(12)*100

ecu.Ecuador.tail()

fecha
2021-Agosto        168.507542
2021-Septiembre    173.213533
2021-Octubre        176.08645
2021-Noviembre     173.361612
2021-Diciembre     190.644754
Name: Ecuador, dtype: object

In [33]:
imaela = round(pd.merge(imaelatam,ecu['Ecuador'],how='left',left_index=True,right_index=True),2)

In [34]:
#   imaela

df = np.array(tb.read_pdf('ecuador.pdf',pages = '5',stream=True,multiple_tables=True,pandas_options={'header':None}))
dff = pd.DataFrame(df.reshape(-1,5)) # en 2022 el reshape debe ser (-1,6)

dff[dff.iloc[3,2].split(' ')] = dff[2].str.split(' ',expand=True)
if dff.iloc[3,3] in range(2020,2040,1):
  dff[int(dff.iloc[3,3])] = dff[3]
  dff[int(dff.iloc[3,4])] = dff[4]
else:
  dff[int(dff.iloc[3,4])] = dff[4]

if dff.iloc[18,0] in ['Ene']:
    ecu = dff.iloc[18:30,[0,-3,-2,-1]].reset_index(drop=True)
else:
    ecu = dff.iloc[19:31,[0,-3,-2,-1]].reset_index(drop=True)
 
ecu['interanualAnterior'] = ecu.iloc[:,-2].astype(float)/ecu.iloc[:,-3].astype(float)*100-100
ecu['Ecuador'] = ecu.iloc[:,-2].astype(float)/ecu.iloc[:,-3].astype(float)*100-100

listames = [meses[1:][i][1] for i in range(len(meses[1:]))]
ecu['mes'] = listames

ecu.index = str(ecu.columns[-4]) + '-' + ecu.mes
ecu.dropna().tail(1)

### **Bolivia**

#### Publicaciones disponibles las fechas:
 
2. 
3. 
4.

In [35]:
urlbol = 'https://www.ine.gob.bo/index.php/estadisticas-economicas/indice-global-de-actividad-economica-igae/#1584545313102-06cf787f-a790'

req = urllib2.Request(url=urlbol, headers=header) 
page = urllib2.urlopen(req).read()

In [36]:
soupbol = BeautifulSoup(page,'lxml')
links = [a['href'] for a in soupbol.select("a[href*=nube.ine.gob.bo/index.php]")] # extraemos links que contengan nube.ine.gob
 # ultimos link disponibles
links # IGAE por actividad economica es el segundo

['https://nube.ine.gob.bo/index.php/s/lhzNu4ISQEuXS1M/download',
 'https://nube.ine.gob.bo/index.php/s/6Szm6RmYSBeRnsD/download',
 'https://nube.ine.gob.bo/index.php/s/qNQexpgoZroGCYG/download',
 'https://nube.ine.gob.bo/index.php/s/vsfGqiIoEXGddwJ/download',
 'https://nube.ine.gob.bo/index.php/s/k0bRzzNhBXb7Ae2/download']

In [37]:
urllib2.urlretrieve(links[1], "Bolivia.xlsx")
#https://nube.ine.gob.bo/index.php/s/6Szm6RmYSBeRnsD/download

('Bolivia.xlsx', <http.client.HTTPMessage at 0x7fcc69a48890>)

In [38]:
df = pd.read_excel('Bolivia.xlsx',header=9)
df.head(2)

,ACTIVIDAD ECONÓMICA,2008,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,2009,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,2010,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,2011,0.33,0.34,...,0.114,0.115,0.116,0.117,0.118,0.119,0.120,2019(p),Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,2020(p),Unnamed: 146,Unnamed: 147,Unnamed: 148,Unnamed: 149,Unnamed: 150,Unnamed: 151,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,2021(p),Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165
0,0,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Ene,Feb,Mar,...,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep
1,INDICE GENERAL,173.164522,167.186087,190.982744,207.586784,206.502976,204.062407,201.217853,193.402673,202.011075,207.979873,199.807696,198.816334,178.470274,173.294198,195.236591,211.581257,210.363103,209.805547,206.190563,199.900466,212.184557,216.774193,209.698656,208.20245,183.889164,177.813348,202.915494,216.940065,219.621267,219.080645,211.36054,206.225131,223.630813,228.557568,220.018383,221.999031,197.77025,190.959467,210.781247,...,316.978159,303.282684,313.250999,332.875262,340.116063,324.799525,330.363965,293.121704,275.729111,310.469355,325.198113,319.065679,327.553037,312.05491,318.071826,340.411294,347.505656,315.462756,342.838053,304.94309,277.363392,297.957751,234.038623,231.827709,266.588831,268.308072,274.025221,311.836456,346.258896,336.1044,340.281821,291.354316,266.681343,312.659752,303.707593,290.251321,298.984049,299.992579,300.749764,321.818286


In [39]:
df.set_index(df.iloc[:,0],inplace=True)
df = df.iloc[:,1:] # eliminar primera columna y ultimas 3 filas
df = df.T.reset_index()

In [40]:
df

ACTIVIDAD ECONÓMICA,index,0,INDICE GENERAL,"AGRICULTURA, PECUARIO, SILVICULTURA, CAZA Y PESCA",PETRÓLEO CRUDO Y GAS NATURAL,MINERALES METÁLICOS Y NO METÁLICOS,INDUSTRIA MANUFACTURERA,"- Alimentos, Bebidas y Tabaco",- Otras Industrias,"ELECTRICIDAD, GAS Y AGUA",CONSTRUCCIÓN,COMERCIO,TRANSPORTE Y ALMACENAMIENTO,COMUNICACIONES,"ESTABLECIMIENTOS FINANCIEROS, SEGUROS, BIENES INMUEBLES",- Servicios Financieros,- Servicios a las Empresas,- Propiedad de Vivienda,SERVICIOS DE LA ADMINISTRACIÓN PÚBLICA,OTROS SERVICIOS(1),Servicios Bancarios Imputados,"Derechos s/Importaciones, IVAnd, IT y otros Imp. Indirectos",Fuente: INSTITUTO NACIONAL DE ESTADÍSTICA,(p): Preliminar,"(1): Incluye las actividades de Restaurantes y Hoteles asi como Servicios Comunales, Sociales, Personales y Domestico"
0,2008,Ene,173.164522,114.1538,302.075282,175.22151,177.093221,204.663583,154.173545,224.204454,114.988011,144.207007,172.283028,505.42049,204.491263,416.422483,208.743136,141.913231,159.254106,169.094149,406.774391,232.087179,NaN,NaN,NaN
1,0,Feb,167.186087,133.012975,272.681746,174.546319,159.269956,207.215561,119.412019,181.51724,84.782734,151.374286,169.463805,365.744586,209.078184,433.149622,215.667673,142.015346,152.864736,169.842304,393.823256,209.998214,NaN,NaN,NaN
2,0.1,Mar,190.982744,235.046083,296.242858,180.243508,165.050937,209.609872,128.008393,181.835758,115.34355,189.427781,175.829663,303.541233,214.161715,403.893294,255.298785,142.092612,154.592727,170.497063,424.687503,227.324648,NaN,NaN,NaN
3,0.2,Abr,207.586784,264.550966,264.612683,196.643475,186.568779,220.510116,158.352811,235.460263,157.670145,206.80088,170.540278,392.217997,237.781321,416.262615,341.379835,142.088875,164.445086,170.883403,447.437168,245.085133,NaN,NaN,NaN
4,0.3,May,206.502976,206.840626,297.10384,204.110062,200.945186,229.765193,176.986661,251.240037,165.345564,200.035958,199.992052,416.599783,250.976093,442.186255,376.225446,142.328559,172.293951,171.649244,402.891577,220.991439,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,Unnamed: 161,May,290.251321,297.058357,346.489047,269.744966,272.773578,340.553254,216.427276,451.051888,257.232839,280.735041,341.742052,627.882851,410.399612,1140.195649,464.941692,177.758408,298.891374,153.331184,1011.645777,304.72693,NaN,NaN,NaN
161,Unnamed: 162,Jun,298.984049,320.499324,355.393647,244.342698,257.770664,309.161796,215.048414,415.141309,318.026413,363.852446,321.3145,515.451074,410.377045,1129.47992,471.353332,178.035394,329.456185,155.096745,1005.804521,302.259594,NaN,NaN,NaN
162,Unnamed: 163,Jul,299.992579,240.988036,333.537651,263.058514,301.675275,350.648399,260.963153,368.265325,417.885566,292.91644,355.737551,715.810933,382.19935,1003.09833,394.278642,199.025691,339.614344,185.846969,1227.882238,393.774127,NaN,NaN,NaN
163,Unnamed: 164,Ago,300.749764,228.208784,322.084882,240.931282,308.858304,373.742012,254.919465,398.946251,477.068164,267.21796,360.797874,706.366004,341.746774,845.192413,333.948107,200.785549,336.262035,209.554549,1115.20166,439.031429,NaN,NaN,NaN


In [41]:
# PARA EXTRAER AÑO DE COLUMNA INDEX
df['index'] = df['index'].astype(str).str.replace('\(p\)','',regex=True)
df['index'] = df['index'].str.replace('^0.[0-9]','nan',regex=True)      # reemplaza los numeros decimales regex que empiece por 0. y le siga cualquier num
df['index'] = df['index'].str.replace('^0$','nan',regex=True)    # reemplaza toda fila con 0 como unico digito
df['index'] = pd.to_numeric(df['index'],errors='coerce').ffill().astype(int)

In [42]:
df['fecha'] = df['index'].astype(str) +'-'+ df[0].str.lower().map(mes3) # columna fecha
df.set_index('fecha',inplace=True)
df.rename(columns={df.columns[2]:'Bolivia'},inplace=True)

In [43]:
imaela = imaela.merge(df['Bolivia'],how='left',left_index=True,right_index=True)

In [44]:
imaela.tail()

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Brasil,México,Perú,Chile,Ecuador,Bolivia
2021-Agosto,106.40,113.32,130.02,257.01,154.33,188.80,322.86,141.08,142.17,107.14,176.10,120.219327,168.507542,300.749764
2021-Septiembre,109.68,114.29,129.23,251.33,150.08,181.01,317.43,140.91,138.86,104.18,178.61,118.388353,173.213533,321.818286
2021-Octubre,112.01,110.98,131.39,267.44,161.08,192.05,366.56,140.53,138.07,108.41,178.91,124.696158,176.08645,NaN
2021-Noviembre,117.14,117.78,135.38,265.80,168.49,204.87,370.39,145.39,139.76,112.75,177.03,130.944118,173.361612,NaN
2021-Diciembre,120.31,NaN,140.26,NaN,NaN,212.34,NaN,NaN,141.46,NaN,194.75,135.293629,190.644754,NaN


### **El Salvador**

#### Publicaciones disponibles:

2. 28 Feb - Lunes
3. 31 Mar
4. 29 Abr
5. 31 May
 


In [45]:
from selenium import webdriver
pagina = 'https://www.bcr.gob.sv/bcrsite/?cdr=174&lang=es'

In [46]:
##### **Otra forma de selenium**
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wdsv = webdriver.Chrome('chromedriver',options=chrome_options)
wdsv.get(pagina)

In [47]:
 #BC El Salvador

# Clicks pertinentes para llegar a la tabla con indices imae SV
wdsv.find_element(By.CLASS_NAME,'linkBtnNaranja').click() # ejecutar consulta

In [48]:
# PARA EXTRAER LOS VALORES DE LOS INDICES

tr = wdsv.find_element(By.ID,'tbl_setValores')
data = tr.find_element(By.TAG_NAME,'tr')
indices_salvador = pd.to_numeric(data.text.split(' ')) 


# PARA EXTRAER LAS FECHAS PARA EL MERGE
soupsv = BeautifulSoup(wdsv.page_source,'html.parser')
anios_sv = soupsv.find_all('td', style='text-align:center')
mes_sv = soupsv.find_all('center')

fecha1 = [anios_sv[i].text for i in range(len(anios_sv))] # lista con años 
fecha2 = [mes_sv[i].text.replace(' ','')[:3] for i in range(1,len(mes_sv))] # lista con meses en formato mmm

fecha=[]
for i in fecha1:
  for j in range(1,13):
    fecha.append(i + "-" + meses_dict[j])

In [49]:
# CREAR DF
elsv = pd.DataFrame(index = fecha,columns=['sv'])
for i in range(len(indices_salvador)):
  elsv.sv[i] = indices_salvador[i]

imaela['El Salvador'] = elsv.loc[imaela.index] # reemplazar valor en la tabla imaela

elsv.loc[imaela.index][-3:]

,sv
2021-Octubre,110.98
2021-Noviembre,117.78
2021-Diciembre,NaN


### **Paraguay**

In [50]:
url = 'https://www.bcp.gov.py/indicador-mensual-de-actividad-economica-del-paraguay-imaep-i471'
urlpy = 'https://www.bcp.gov.py/anexo-estadistico-del-informe-economico-i365'
#the URL where you are requesting at
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
req = urllib2.Request(url=urlpy, headers=header) 
page = urllib2.urlopen(req).read()

In [51]:
wdpy = webdriver.Chrome('chromedriver',options=chrome_options)
wdpy.get(urlpy)

In [52]:
souppy = BeautifulSoup(wdpy.page_source,'lxml')
links = [a['href'] for a in souppy.select("a[href*=.xlsx]")] # extraemos links que contengan .xlsx
 # ultimos link disponibles
links

['userfiles/files/Anexo_Estad%C3%ADstico%20del%20Informe%20Econ%C3%B3mico_14_02_2022.xlsx']

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context
url2py = 'https://www.bcp.gov.py/' + links[0]
req2 = requests.get(url2py, headers = header)  # nueva forma de descargar excel file de pagina de py
#urllib2.urlretrieve(url2py, "py.xlsx")

In [ ]:
import datetime
dfpy = pd.read_excel(req2.content,sheet_name='CUADRO 9',header=9).iloc[:-3,1:] # con limpieza iloc CUADRO 9 es IMAEP
dfpy.rename(columns={dfpy.columns[1]:"Paraguay"},inplace=True)

In [ ]:
# FECHA COLUMNA
dfpy['A'] = pd.DatetimeIndex(dfpy[dfpy.columns[0]]).year
dfpy['M'] = pd.DatetimeIndex(dfpy[dfpy.columns[0]]).month.map(meses_dict)
dfpy['fecha'] = dfpy.A.astype(str) + "-" + dfpy.M.astype(str)
dfpy.set_index('fecha',inplace=True)

In [ ]:
imaela = imaela.merge(dfpy['Paraguay'],how='left',left_index=True,right_index=True)

In [ ]:
imaela


url2 = 'https://www.bcp.gov.py/' + links[0] # -(numero mes)
opener = urllib2.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib2.install_opener(opener)
urllib2.urlretrieve(url2, "py.pdf")




'With pdf file

with fitz.open("py.pdf") as doc:
    text = ""
    for page in doc:
        text += page.get_text()

fechapy = text.split(' |',2)[1] # con el espacio
fechapy = fechapy.split(' ')[1] +'-'+fechapy.split(' ')[0]

Debido a que las tasas son iguales en Enero

if 'Enero' not in fechapy:
  valorpyinteranual = re.findall(r'[0-9]+,[0-9]+',text)[0] # para obtener tasa interanual          el regex sirve para buscar los registros numericos separados por coma
  valorpyacum = re.findall(r'[0-9]+,[0-9]+',text)[1] # para obtener tasa acumulada interanual
  valorpyinteranual = float(valorpyinteranual.replace(',','.'))
  valorpyacum = float(valorpyacum.replace(',','.'))
else:
   valorpyinteranual = re.findall(r'[0-9]+,[0-9]+',text)[0]
   valorpyinteranual = float(valorpyinteranual.replace(',','.'))
   valorpyacum = valorpyinteranual

print(fechapy)
print(valorpyinteranual,' Tasa interanual')
print(valorpyacum, ' Tasa acumulada interanual')


py = pd.DataFrame({'Fecha':['2021-Enero','2021-Febrero','2021-Marzo','2021-Abril','2021-Mayo','2021-Junio','2021-Julio','2021-Agosto','2021-Septiembre','2021-Octubre','2021-Noviembre','2021-Diciembre'],
                   'Paraguay_a':[-5.6,-3.0,0.7,4.7,6.3,6.3,6.0,5.7,0,0,0,0],
                   'Paraguay_i':[-5.6,-0.5,8.0,19.3,12.4,6.6,3.0,4.2,0,0,0,0]})
py.index = py.Fecha
dic = {'Fecha':fechapy,'Paraguay_a':valorpyacum,'Paraguay_i':valorpyinteranual}
py.loc[fechapy] = [fechapy,valorpyacum,valorpyinteranual]
py

In [ ]:
imaela['a'] = [imaela.index.str.split('-')[i][0] for i in range(len(imaela.index))]
imaela = imaela.apply(pd.to_numeric)

In [ ]:
# Crecimiento interanual
imaela1 = imaela.pct_change(12,fill_method=None)*100
imaela1.dropna(how='all',inplace=True) # NAs drop
imaela1.drop('a',axis=1,inplace=True)

In [ ]:
# Crecimiento acumulado interanual
imaela2 = imaela.groupby('a').cumsum().pct_change(12,fill_method=None)*100
imaela2.dropna(how='all',inplace=True) # NAs drop

In [ ]:
#!pip install selenium
#!apt-get update # to update ubuntu to correctly run apt install
#!apt install chromium-chromedriver
#!cp /usr/lib/chromium-browser/chromedriver /usr/bin
#import sys
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
#from selenium import webdriver
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument('--headless')
#chrome_options.add_argument('--no-sandbox')
#chrome_options.add_argument('--disable-dev-shm-usage')
#wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

### **Colombia**

#### Publicaciones disponibles las fechas: 
1. 15 Feb
2. 18 Mar
3. 18 Abr
4. 16 May
5. 17 Jun
6. 18 Jul
7. 16 Ago
8. 19 Sep

wd.get(url1)
wd.page_source
!pip install fake_useragent
from fake_useragent import UserAgent
ua = UserAgent()
print(ua.chrome)
print(ua.random)
url = 'https://www.dane.gov.co/index.php/estadisticas-por-tema/cuentas-nacionales/indicador-de-seguimiento-a-la-economia-ise/historicos-ise-comunicados-y-boletines'
url1 = 'https://www.dane.gov.co/index.php/estadisticas-por-tema/cuentas-nacionales/indicador-de-seguimiento-a-la-economia-ise'

header_col = {'authority': 'www.dane.gov.co', 'method': 'GET', 'path': '/index.php/estadisticas-por-tema/cuentas-nacionales/indicador-de-seguimiento-a-la-economia-ise',
'scheme': 'https', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'referer':'https://www.dane.gov.co/index.php/estadisticas-por-tema/cuentas-nacionales/indicador-de-seguimiento-a-la-economia-ise',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'es-ES,es;q=0.9',
'cache-control': 'max-age=0',
'cookie': '_ga=GA1.3.1467282597.1625839173; _ga_T3L41X6043=GS1.1.1632235276.4.0.1632235284.0; ec1d9c9ecb5c2f4ef7998e3e52b19fdf=bvqfvrh7p0e0kgp1vf8bg29s68; _gid=GA1.3.2105323501.1637672614',
'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': "Windows",
'sec-fetch-dest': 'document',
'sec-fetch-mode': 'navigate',
'sec-fetch-site': 'none',
'sec-fetch-user': '?1',
'upgrade-insecure-requests':' 1',
'user-agent': str(ua.random)}

req = urllib2.Request(url=url1, headers=header_col) 
page = urllib2.urlopen(req).read()

soupco = BeautifulSoup(page,'lxml')
links = [a['href'] for a in soupco.select("a[href*=21.xlsx]")] # extraemos links que contengan 21.xlsx 
 #### ultimos link disponibles
links
url2 = 'https://www.dane.gov.co' + links[0] # si esta el nuevo mes restarle 1 al indice de links
url3 = 'https://www.dane.gov.co/files/investigaciones/boletines/pib/Anex_ISE_9actividades_sep_21.xlsx'
####urllib2.urlretrieve(url2, "colombia.xlsx")
urllib2.urlretrieve(url3, "colombia.xlsx",)
df = pd.read_excel('colombia.xlsx',sheet_name='Cuadro 1',header=9)[0:16].T
df.columns = df.iloc[0]
df = df.iloc[:,[0,1,-1]].reset_index(drop=True)[1:]
df.tail(1)

df['ano'] = df.Concepto.ffill().astype(str).str.replace('pr','').str.replace('p','')
df['Colombia'] = round(pd.to_numeric(df.loc[:,df.columns[-2]]),2)
df.index = df.ano.astype(str) + '-' + df.iloc[:,1].astype(str) 

imaela = imaela.merge(df['Colombia'],how='left',left_index=True,right_index=True)

##### PDF

df = np.array(tb.read_pdf('colombia.pdf',pages = '17',stream=True,multiple_tables=True,pandas_options={'header':None}))
dco = pd.DataFrame(df.reshape(-1,4))
dco = dco.loc[:,0:2]
dco.tail(1)

dco[3] = dco[2].str.split(' ',expand=True)[0] # Tomando las tasas de crecimiento de la tabla pdf
dco.rename(columns={1:'Colombia'},inplace=True)
dco['Colombia1'] = pd.to_numeric(dco[3].str.replace(',','.'),errors='coerce')  #Convirtiendo la data a numero
dco['Colombia'] = pd.to_numeric(dco['Colombia'].str.replace(',','.'),errors='coerce')

split = dco[0].str.split(' - ')     # extraer mes 
fech = []
for i in range(len(split)):
  if type(split[i]) == float:
    fech.append('0')
  elif len(split[i]) == 1 :
    fech.append(str(split[i][0]))
  else:
    fech.append(str(split[i][1].capitalize()))

anof=[]                          # extraer año
for i in range(len(split)):
  if type(split[i]) == float:
    anof.append('0')
  elif len(split[i]) > 1 :
    anof.append(str(split[i][0]))
  else:
    anof.append(np.nan)

dco['mes'] = fech
dco['ano'] = anof
dco['ano'] = dco.ano.replace(['p','r',' '],['','',''],regex=True).ffill()
dco.index = dco['ano'] + '-' + dco['mes'] # creamos indice

## imae


In [ ]:
# imaela.drop('a',axis=1,inplace=True)
imaela.drop('a',axis=1,inplace=True)

In [55]:
imaela.tail(3)

,Costa Rica,El Salvador,Guatemala,Honduras,Nicaragua,República Dominicana,Panamá,Argentina,Brasil,México,Perú,Chile,Ecuador,Bolivia
2021-Octubre,112.01,110.98,131.39,267.44,161.08,192.05,366.56,140.53,138.07,108.41,178.91,124.696158,176.08645,NaN
2021-Noviembre,117.14,117.78,135.38,265.80,168.49,204.87,370.39,145.39,139.76,112.75,177.03,130.944118,173.361612,NaN
2021-Diciembre,120.31,NaN,140.26,NaN,NaN,212.34,NaN,NaN,141.46,NaN,194.75,135.293629,190.644754,NaN


In [57]:
#@title Descarga a Excel
writer = pd.ExcelWriter('imae_latam.xlsx', engine='xlsxwriter')

#imaela1.to_excel(writer, sheet_name='Interanual')
#imaela2.to_excel(writer, sheet_name='Acumulado')
imaela.to_excel(writer, sheet_name='Indices - Serie Original')
writer.save()

files.download("imae_latam.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title Bar
f, ax = plt.subplots(figsize=(20, 8))
sns.barplot(data=imaela.iloc[-1:,:]);

In [ ]:
#@title Interanual Mes Anterior
f, ax = plt.subplots(figsize=(18, 8))
df = pd.melt(imaela1.iloc[-2:-1,:]).sort_values('value',ascending=False) 
sns.barplot(data = df ,x='value',y='variable',
                palette=['r' if y =='República Dominicana' else '#0343DF' for y in df.variable])
for p in ax.patches:
    width = p.get_width()    # get bar length
    ax.text(width+0.2,       # set the text at 0.2 unit right of the bar
            p.get_y() + p.get_height() / 2, # get Y coordinate + X coordinate / 2
            '{:1.2f}'.format(width), # set variable to display, 2 decimals
            ha = 'left',   # horizontal alignment
            va = 'center', weight="bold") # vertical alignment
plt.title('IMAE LATAM \n Crecimiento Interanual (%) {}'.format(imaela.index[-2]),weight='bold');

In [ ]:
#@title Acumulado Interanual Mes Anterior
f, ax = plt.subplots(figsize=(18, 8))
df = pd.melt(imaela2.iloc[-2:-1,:]).sort_values('value',ascending=False) 
sns.barplot(data = df ,x='value',y='variable',
                palette=['r' if y =='República Dominicana' else '#0343DF' for y in df.variable])
for p in ax.patches:
    width = p.get_width()    # get bar length
    ax.text(width+0.2,       # set the text at 0.2 unit right of the bar
            p.get_y() + p.get_height() / 2, # get Y coordinate + X coordinate / 2
            '{:1.2f}'.format(width), # set variable to display, 2 decimals
            ha = 'left',   # horizontal alignment
            va = 'center', weight="bold") # vertical alignment
plt.title('IMAE LATAM \n Crecimiento Acumulado Interanual (%) a {}'.format(imaela.index[-2]) ,weight='bold');

In [ ]:
#@title Interanual Ultimo Mes
f, ax = plt.subplots(figsize=(18, 8))
df = pd.melt(imaela1.iloc[-1:,:].dropna(axis=1)).sort_values('value',ascending=False) 
sns.barplot(data = df ,x='value',y='variable',
                palette=['r' if y =='República Dominicana' else '#0343DF' for y in df.variable])
for p in ax.patches:
    width = p.get_width()    # get bar length
    ax.text(width+0.2,       # set the text at 0.2 unit right of the bar
            p.get_y() + p.get_height() / 2, # get Y coordinate + X coordinate / 2
            '{:1.2f}'.format(width), # set variable to display, 2 decimals
            ha = 'left',   # horizontal alignment
            va = 'center', weight="bold") # vertical alignment
plt.title('IMAE LATAM \n Crecimiento Interanual (%) {}'.format(imaela.index[-1]),weight='bold')
;

In [ ]:
#@title Acumulado Interanual Ultimo Mes
f, ax = plt.subplots(figsize=(18, 8))
df = pd.melt(imaela2.iloc[-1:,:].dropna(axis=1)).sort_values('value',ascending=False) 
sns.barplot(data = df ,x='value',y='variable',
                palette=['r' if y =='República Dominicana' else '#0343DF' for y in df.variable])
for p in ax.patches:
    width = p.get_width()    # get bar length
    ax.text(width+0.2,       # set the text at 0.2 unit right of the bar
            p.get_y() + p.get_height() / 2, # get Y coordinate + X coordinate / 2
            '{:1.2f}'.format(width), # set variable to display, 2 decimals
            ha = 'left',   # horizontal alignment
            va = 'center', weight="bold") # vertical alignment
plt.title('IMAE LATAM \n Crecimiento Acumulado Interanual (%) a {}'.format(imaela.index[-1]) ,weight='bold');